In [31]:
import time
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import random

In [32]:

def get_quantile_format(fctdf,model_name):
    qfctdf=pd.DataFrame()
    map_col={}
    for col in fctdf.columns:
        if col == "unique_id":
            continue
    
        quantile = None
        if "lo" in col:
            number = int(col[-2:])
            alpha = 1 - (number / 100)
            quantile = round(alpha / 2, 3)
        elif "hi" in col:
            number = int(col[-2:])
            alpha = 1 - (number / 100)
            quantile = round(1 - (alpha / 2), 3)
        elif col.endswith("-median") or col == model_name:
            quantile = 0.5
        else:
            continue
        map_col[col]=quantile
    
    qfctdf=fctdf.set_index('ds').melt(id_vars='unique_id',var_name='output_type_id',ignore_index=False)
    qfctdf.output_type_id=qfctdf.output_type_id.map(map_col)
    qfctdf=qfctdf.reset_index()
    qfctdf=qfctdf.rename(columns={'ds':'target_end_date','Reference Date':'reference_date'})
    # qfctdf.loc[:,'reference_date']=reference_date

    uid=fctdf.unique_id.unique()[0]
    gt_series = df_test_all[df_test_all.unique_id==uid]#.loc[df_col.index]

    targ_dates=fctdf['ds'].values
    gt = gt_series[gt_series.ds.isin(targ_dates)]
    gt=gt.rename(columns={'ds':'target_end_date','y':'GT'})

    qfctdf=qfctdf.merge(gt,on=['target_end_date','unique_id'])
    uid=fctdf.unique_id.unique()
    # qfctdf.loc[:,'unique_id']=uid
    return qfctdf
def get_test_fcts(df_test_all,model_name,saved_path):
    nf2 = NeuralForecast.load(path=saved_path)
    all_fctdf=pd.DataFrame()
    uids=df_test_all.unique_id.unique()
    for uid in uids:
        df_test=df_test_all[df_test_all.unique_id==uid]
        for i in range(8,56):
            test_data=df_test.iloc[:i]
            # uid = test_data.unique_id.unique()[0]
            reference_date = test_data.index[-1]
            fctdf=nf2.predict(test_data)
            qfctdf=get_quantile_format(fctdf,model_name=model_name)
            qfctdf.loc[:,'model']=model_name
            qfctdf.loc[:,'reference_date']=reference_date
            all_fctdf=pd.concat([all_fctdf,qfctdf])
    return all_fctdf

def get_wis(all_fctdf):
    results = []
    grouped = all_fctdf.groupby(["unique_id", "reference_date", "target_end_date"])
    
    for (uid, ref_date, tgt_date), group in grouped:
        gt = group["GT"].iloc[0]
        group = group.sort_values("output_type_id")
    
        if not np.any(np.isclose(group["output_type_id"], 0.5)):
            continue
    
        median_pred = group[np.isclose(group["output_type_id"], 0.5)]["value"].iloc[0]
        ae = abs(median_pred - gt)
    
        lo = group[group["output_type_id"] < 0.5]
        hi = group[group["output_type_id"] > 0.5][::-1].reset_index(drop=True)
    
        wis_components = []
        for i in range(min(len(lo), len(hi))):
            alpha = hi.iloc[i]["output_type_id"] - lo.iloc[i]["output_type_id"]
            lo_pred, hi_pred = lo.iloc[i]["value"], hi.iloc[i]["value"]
            interval_score = (
                hi_pred - lo_pred
                + (2 / alpha) * (lo_pred - gt) * (gt < lo_pred)
                + (2 / alpha) * (gt - hi_pred) * (gt > hi_pred)
            )
            wis_components.append(interval_score)
    
        wis = (ae + np.sum(wis_components)) / (1 + len(wis_components))
        results.append({
            "Unique_id": uid,
            "Reference Date": ref_date,
            "Target End Date": tgt_date,
            "GT": gt,
            "WIS": wis})
    return pd.DataFrame(results)

In [33]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("../outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [34]:
seed_value=256
random.seed(seed_value)
N=100
sub_test_indices=random.sample(list(test_indices),N)

In [35]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in sub_test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [36]:
df_test_all.shape

(6000, 3)

In [37]:
# # Load wide-format data
# df_wide = pd.read_csv("/sfs/gpfs/tardis/project/bii_nssac/people/aniadiga/forecast/gits/benchmark_MIDAS/data/test.csv")
# value_cols = [str(i) for i in range(60)]
# start_dates = pd.to_datetime(df_wide["start_date"])
# series_values = df_wide[value_cols].astype(float).fillna(0)

# # Convert to long format
# records = []
# for i, (start_date, row) in enumerate(zip(start_dates, series_values.values), start=1):
#     adjusted_start = start_date - pd.Timedelta(weeks=4)
#     dates = pd.date_range(start=adjusted_start, periods=60, freq="W-SAT")
#     for t, value in enumerate(row):
#         records.append({"unique_id": f"Y_{i}", "ds": dates[t], "y": value})

# df_long = pd.DataFrame(records)

In [38]:
uids=df_test_all.unique_id.unique()
m='AutoARIMA'
freq='W-SAT'
h=4
allforecasts=pd.DataFrame()
for uid in uids:
    filt_data=df_test_all[df_test_all.unique_id==uid]
    # allqfctdf=pd.DataFrame()
    # for i in range(8,56):
    for i in range(8,56):
        data=filt_data.iloc[:i]
        reference_date = data.iloc[-1]['ds']
        sf=StatsForecast(models=[AutoARIMA(season_length=1)], freq=freq, n_jobs=-1)
        sf.fit(data)
        forecast = sf.predict(h=h, level = [10,20,30,40,50,60,70,80, 85,90,95])
        # reference_date=data.iloc[-1]['ds']
        # forecast.loc[:,'reference_date']=reference_date
        qfctdf=get_quantile_format(forecast,model_name=m)
    
        qfctdf.loc[:,'reference_date']=reference_date
        qfctdf.loc[:,'model']=m
    
        # allqfctdf=pd.concat([allforecasts,qfctdf])
        allforecasts=pd.concat([allforecasts,qfctdf])
        
allforecasts.to_csv(f'../../../output/forecasts/{m}_{N}_{seed_value}.csv',index=None)
        
evaldf=get_wis(allforecasts)
evaldf.loc[:,'model']=m
        #             evaldf.loc[:,'model']=m
evaldf.to_csv(f'../../../eval/eval_fcts/{m}_{N}_{seed_value}.csv',index=None)
        # qfctdf.to_csv('')

In [130]:
allforecasts.unique_id.unique()

array(['Y_5607', 'Y_5978'], dtype=object)

In [133]:
get_wis(allforecasts)

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_5607,2023-08-26,2022-10-08,29725.0,73538.752747
1,Y_5607,2023-08-26,2022-10-15,38997.0,119944.976376
2,Y_5607,2023-08-26,2022-10-22,46568.0,111392.923300
3,Y_5607,2023-08-26,2022-10-29,42898.0,104229.404280
4,Y_5607,2023-08-26,2022-11-05,31562.0,117022.286245
...,...,...,...,...,...
97,Y_5978,2023-08-26,1944-12-02,0.0,6.248023
98,Y_5978,2023-08-26,1944-12-09,0.0,6.112594
99,Y_5978,2023-08-26,1944-12-16,0.0,6.287816
100,Y_5978,2023-08-26,1944-12-23,0.0,6.428637


In [128]:
2*23*len(range(8,56))*4

8832

In [106]:
gt_series = df_test_all[df_test_all.unique_id==uid]#.loc[df_col.index]
# gt_series.index=pd.to_datetime(gt_series.index)

targ_dates=forecast['ds'].values
gt=gt_series.loc[targ_dates][['y']].reset_index()
gt=gt.rename(columns={'date':'target_end_date','y':'GT'})

KeyError: "None of [DatetimeIndex(['2023-09-02', '2023-09-09', '2023-09-16', '2023-09-23'], dtype='datetime64[ns]', freq=None)] are in the [index]"

In [69]:
gt_series = df_test_all[df_test_all.unique_id==uid]

In [115]:
qfctdf

,target_end_date,unique_id,output_type_id,value,GT,reference_date,model
0,2023-09-02,Y_5607,NaN,1231.013469,1231.0,2023-08-26,ARIMA
1,2023-09-09,Y_5607,NaN,1231.027385,1231.0,2023-08-26,ARIMA
2,2023-09-16,Y_5607,NaN,1231.035386,0.0,2023-08-26,ARIMA
3,2023-09-23,Y_5607,NaN,1231.036434,0.0,2023-08-26,ARIMA
4,2023-09-02,Y_5607,0.025,-1823.793095,1231.0,2023-08-26,ARIMA
...,...,...,...,...,...,...,...
87,2023-09-23,Y_5607,0.950,17604.945532,0.0,2023-08-26,ARIMA
88,2023-09-02,Y_5607,0.975,4285.820034,1231.0,2023-08-26,ARIMA
89,2023-09-09,Y_5607,0.975,8960.669209,1231.0,2023-08-26,ARIMA
90,2023-09-16,Y_5607,0.975,14970.886566,0.0,2023-08-26,ARIMA


In [119]:
get_wis(qfctdf)

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_5607,2023-08-26,2023-09-02,1231.0,2641.099930
1,Y_5607,2023-08-26,2023-09-09,1231.0,6682.830596
2,Y_5607,2023-08-26,2023-09-16,0.0,12565.198961
3,Y_5607,2023-08-26,2023-09-23,0.0,16970.996169


,unique_id,ds,y
55,Y_5607,2023-09-02,1231.0
56,Y_5607,2023-09-09,1231.0
57,Y_5607,2023-09-16,0.0
58,Y_5607,2023-09-23,0.0


In [8]:
class FixedARIMAProcessor:
    def __init__(self):
        self.forecasts = []
        self.eval_pairs = []
        self.unique_ids = []
        self.dates = []

        self.maes = []
        self.mses = []
        self.mapes = []
        self.nmses = []

        self.metrics_df = pd.DataFrame(columns=["Reference Date", "MAE", "MSE", "MAPE", "NMSE"])
        self.display_df = pd.DataFrame(columns=["Unique_id", "Reference Date", "Target End Date", "GT", "Quantile", "Prediction"])
        self.color_mapping = {}

    def create_fixed_model(self, df_long, h, freq="W-SAT", level=[80, 95], season_length=1):
        df_fit = df_long.groupby("unique_id").apply(lambda g: g.iloc[:-h]).reset_index(drop=True)
        df_truth = df_long.groupby("unique_id").apply(lambda g: g.iloc[-h:]).reset_index(drop=True)

        start = time.time()
        self.sf = StatsForecast(models=[AutoARIMA(season_length=season_length)], freq=freq, n_jobs=-1)
        self.sf.fit(df_fit)
        print(f"ARIMA fit time: {time.time() - start:.2f} sec")

        forecast = self.sf.predict(h=h, level=level)
        forecast.set_index(["unique_id", "ds"], inplace=True)
        df_truth.set_index(["unique_id", "ds"], inplace=True)

        print("Processing forecasts per series...")
        for uid in tqdm(df_fit["unique_id"].unique(), desc="Fitting per series"):
            f = forecast.loc[uid].copy()
            f["unique_id"] = uid
            t = df_truth.loc[uid]
            self.forecasts.append(f)
            self.eval_pairs.append((f, t))
            self.dates.append(df_fit[df_fit["unique_id"] == uid]["ds"].max().strftime("%Y-%m-%d"))
            self.unique_ids.append(uid)

    def calculate_metrics(self):
        for forecast_df, truth_df in self.eval_pairs:
            y_true = truth_df.iloc[:, 0]
            y_pred = forecast_df.iloc[:, 0]

            mae = mean_absolute_error(y_true, y_pred)
            mse = mean_squared_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true, y_pred)
            nmse = mse / np.var(y_true)

            self.maes.append(mae)
            self.mses.append(mse)
            self.mapes.append(mape)
            self.nmses.append(nmse)

    def create_metrics_df(self):
        self.metrics_df = pd.DataFrame({
            "Reference Date": self.dates,
            "MAE": self.maes,
            "MSE": self.mses,
            "MAPE": self.mapes,
            "NMSE": self.nmses,
        })

    def create_display_df(self):
        records = []
        print("Generating display DataFrame...")
        for i in tqdm(range(len(self.forecasts)), desc="Building display_df"):
            forecast_df = self.forecasts[i]
            reference_date = self.dates[i]
            unique_id = self.unique_ids[i]
            truth_series = self.eval_pairs[i][1].iloc[:, 0]

            for col in forecast_df.columns:
                if col == "unique_id":
                    continue
                if "lo" in col or "hi" in col:
                    number = int(col.split("-")[-1])
                    alpha = 1 - (number / 100)
                    quantile = 1 - (alpha / 2) if "hi" in col else alpha / 2
                elif col == "AutoARIMA":
                    quantile = 0.5
                else:
                    continue

                preds = forecast_df[col]
                for idx, pred in preds.items():
                    records.append({
                        "Unique_id": unique_id,
                        "Reference Date": reference_date,
                        "Target End Date": idx,
                        "GT": truth_series.get(idx, np.nan),
                        "Quantile": quantile,
                        "Prediction": pred
                    })

        self.display_df = pd.DataFrame(records).sort_values(
            by=["Unique_id", "Reference Date", "Target End Date", "GT", "Quantile"]
        ).reset_index(drop=True)

    def compute_wis(self):
        display_df = self.display_df.sort_values(by=["Unique_id", "Reference Date", "Target End Date", "Quantile"])
        records = []

        print("Computing WIS for each forecasted point...")
        grouped = display_df.groupby(["Unique_id", "Reference Date", "Target End Date"])

        for (uid, ref_date, tgt_date), group in tqdm(grouped, desc="Computing WIS"):
            gt = group["GT"].values[0]
            predictions = group.set_index("Quantile")["Prediction"]
            quantiles = sorted(predictions.index)

            pairs = list(zip(quantiles[:len(quantiles)//2], quantiles[::-1][:len(quantiles)//2]))
            wis_components = []

            for lo_q, hi_q in pairs:
                lo_pred = predictions.get(lo_q, np.nan)
                hi_pred = predictions.get(hi_q, np.nan)
                alpha = hi_q - lo_q

                interval_score = (
                    (hi_pred - lo_pred)
                    + (2 / alpha) * (lo_pred - gt) * (gt < lo_pred)
                    + (2 / alpha) * (gt - hi_pred) * (gt > hi_pred)
                )
                wis_components.append(interval_score)

            ae = abs(gt - predictions[0.5]) if 0.5 in predictions else np.nan
            wis = (ae + sum(wis_components)) / (1 + len(wis_components))

            records.append({
                "Unique_id": uid,
                "Reference Date": ref_date,
                "Target End Date": tgt_date,
                "GT": gt,
                "WIS": wis
            })

        return pd.DataFrame(records)

In [10]:
processor.create_fixed_model(df_long=df_long, h=4, freq="W-SAT", season_length=60, level=[10,20,30,40,50,60,70,80,85,90,95])

/tmp/ipykernel_986472/1103351850.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_fit = df_long.groupby("unique_id").apply(lambda g: g.iloc[:-h]).reset_index(drop=True)
/tmp/ipykernel_986472/1103351850.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_truth = df_long.groupby("unique_id").apply(lambda g: g.iloc[-h:]).reset_index(drop=True)


ARIMA fit time: 33.24 sec
Processing forecasts per series...


Fitting per series: 100%|██████████| 3240/3240 [00:33<00:00, 97.22it/s] 


In [11]:
processor.create_display_df()

Generating display DataFrame...


Building display_df: 100%|██████████| 3240/3240 [00:05<00:00, 634.26it/s]


In [13]:
arimadf=processor.display_df

In [22]:
arimadf.loc[arimadf['value']<0,'value']=0
arimadf.loc[:,'value']=arimadf['value'].astype(int)

In [16]:
arimadf=arimadf.rename(columns={'Unique_id':'outbreak_id','Reference Date':'reference_date','Target End Date':'target_end_date','Quantile':'output_type_id','Prediction':'value'})

In [18]:
arimadf.loc[:,'model']='ARIMA'


In [25]:
out_folder='/sfs/gpfs/tardis/project/bii_nssac/people/aniadiga/forecast/gits/benchmark_MIDAS/output/ARIMA/'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
arimadf.to_csv(out_folder+'/'+'submission.csv',index=None)

In [10]:
wis_df = processor.compute_wis()

Computing WIS for each forecasted point...


Computing WIS: 100%|██████████| 43196/43196 [00:16<00:00, 2609.33it/s]


In [11]:
wis_df

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_1,2025-06-14,2025-06-21,0.0,8.712642
1,Y_1,2025-06-14,2025-06-28,0.0,12.321537
2,Y_1,2025-06-14,2025-07-05,0.0,15.090739
3,Y_1,2025-06-14,2025-07-12,0.0,17.425285
4,Y_10,2016-10-01,2016-10-08,0.0,0.234415
...,...,...,...,...,...
43191,Y_9998,1954-12-25,1955-01-22,0.0,1.021407
43192,Y_9999,1955-03-19,1955-03-26,0.0,7.968953
43193,Y_9999,1955-03-19,1955-04-02,0.0,10.187655
43194,Y_9999,1955-03-19,1955-04-09,0.0,12.875487


In [12]:
mean_wis = np.mean(wis_df['WIS'].values)

In [27]:
mean_wis

1725.6276646823117

In [13]:
wis_dfs = [wis_df.iloc[i::4].reset_index(drop=True) for i in range(4)]

In [17]:
wis_dfs[0]

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_1,2025-06-14,2025-06-21,0.0,8.712642
1,Y_10,2016-10-01,2016-10-08,0.0,0.234415
2,Y_100,1945-03-31,1945-04-07,0.0,7.023985
3,Y_1000,2022-01-01,2022-01-08,0.0,2589.278167
4,Y_10000,1957-02-23,1957-03-02,0.0,8.503033
...,...,...,...,...,...
10794,Y_9995,1952-04-05,1952-04-12,1.0,3.566430
10795,Y_9996,1953-03-07,1953-03-14,1.0,12.124884
10796,Y_9997,1954-02-20,1954-02-27,0.0,13.431079
10797,Y_9998,1954-12-25,1955-01-01,0.0,0.704451


In [21]:
np.mean(wis_dfs[3]['WIS'].values)

2457.1950602027355

In [28]:
wis_df.to_csv('ARIMA_DF_WIS.csv')